<a href="https://colab.research.google.com/github/kevinautomation/cohort_project_ask_red/blob/main/reddit_expert_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
trained_model_id = "kevinautomation/mistral-7b-instruct-tune-project_ask_reddit"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map='auto'
)

trained_model = AutoModelForCausalLM.from_pretrained(
    trained_model_id,
    quantization_config=bnb_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


In [ ]:
def generate_response(prompt, model, tokenizer):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "").replace("\n","")

In [ ]:
prompt_template = "### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Input:\n{input}	\n\n### Response:"
input = "why are the skins of fruit more nutritious than the insides?"
prompt = prompt_template.format(prompt_template, input=input)

In [ ]:
generate_response(prompt,
                  base_model,
                  tokenizer)

"<s> The answer to that question is a fascinating one (as one might suspect). For the most part, that assumption is true. But for an interesting case it is not, such as that of the avocado.### Description of approach:I found answer.io which asks real-world question and answers them very accurately. I wrote a Python script to iterate through the questions and answers to extract the above answer, and then post it through Facebook's Graph API endpoint /send for posts so that it would appear as a post (instead of just text).### Result:To make sure that facebook didn't think it was spam, I changed the user_id and page_id after running it, and tried adding links to make it seem legitimate. I ended up posting this onto my Facebook wall but am unsure whether the bot would continue to answer more questions.</s>"

In [ ]:
generate_response(prompt,
                  trained_model,
                  tokenizer)

'<s> The outer layer is sometimes called the exocarp (skins on plum) or epicarp (skin on a berry) and they both can contain various compounds called polyphenols but mainly compounds called anthocyanins. These compounds give the red, purple, and darker colors and are responsible for most of the positive health effects. As for the fruit nutrients they mostly are found within the edible flesh, which are usually water soluble and are affected by light. This is why they need to be eaten as soon as possible and not stored in a fridge for too long, as it degrades the nutrients. Some nutrients like phytochrome, are also damaged by light. This also explains why many nutrients are found more deeply in the fruit and not on the outer layer.</s>'